In [1]:
#Importing Libraries for reading a graph
from networkx.readwrite import json_graph
import json 
import os
from gerrychain import Graph
def read_graph_from_json(json_file):
    with open(json_file) as x:
        data = json.load(x)
    return json_graph.adjacency_graph(data)

In [2]:
# Reading Arkansas population file with no of counties
filepath = '/Users/ujwalkumarmellacheruvu/Public/REDISTRICTING'
filename = 'AR_county.json'
# Construct the file path 
New_path = os.path.join(filepath,filename)
# Frame the graph 
G = Graph.from_json(New_path)

In [3]:
## Let's impose a 1% population deviation (+/-0.5%)
Standard_deviation = 0.01

import math 
# No of districts in arkansas
k = 4
#Summing up the total population in counties 
total_population = sum(G.nodes[node]['POP100'] for node in G.nodes)

# Equations for bounds
L = math.ceil((1-Standard_deviation/2)*total_population/k)
U = math.floor((1+Standard_deviation/2)*total_population/k)

print("Using L =",L,"and U =",U,"and K =",k)

Using L = 749117 and U = 756645 and K = 4


In [4]:
#Get the population for each county
for node in G.nodes:
    name =  name = G.nodes[node]['NAME20']
    population = G.nodes[node]['POP100']
    print("Node",node, name," 2021 CENSUS of",population)


Node 0 Franklin  2021 CENSUS of 17097
Node 1 Crawford  2021 CENSUS of 60133
Node 2 Jackson  2021 CENSUS of 16755
Node 3 Clay  2021 CENSUS of 14552
Node 4 Faulkner  2021 CENSUS of 123498
Node 5 Baxter  2021 CENSUS of 41627
Node 6 Little River  2021 CENSUS of 12026
Node 7 Boone  2021 CENSUS of 37373
Node 8 Ashley  2021 CENSUS of 19062
Node 9 Desha  2021 CENSUS of 11395
Node 10 St. Francis  2021 CENSUS of 23090
Node 11 Montgomery  2021 CENSUS of 8484
Node 12 Sharp  2021 CENSUS of 17271
Node 13 Greene  2021 CENSUS of 45736
Node 14 Woodruff  2021 CENSUS of 6269
Node 15 White  2021 CENSUS of 76822
Node 16 Lee  2021 CENSUS of 8600
Node 17 Conway  2021 CENSUS of 20715
Node 18 Howard  2021 CENSUS of 12785
Node 19 Nevada  2021 CENSUS of 8310
Node 20 Pulaski  2021 CENSUS of 399125
Node 21 Grant  2021 CENSUS of 17958
Node 22 Benton  2021 CENSUS of 284333
Node 23 Madison  2021 CENSUS of 16521
Node 24 Dallas  2021 CENSUS of 6482
Node 25 Cleveland  2021 CENSUS of 7550
Node 26 Crittenden  2021 CENSUS 

In [5]:
# Begin with model
import gurobipy as gp
from gurobipy import GRB

# create model 
m = gp.Model()

# create variables
x = m.addVars(G.nodes, k, vtype=GRB.BINARY) # x[i,j] equals one when county i is assigned to district j
y = m.addVars(G.edges, vtype=GRB.BINARY)    # y[u,v] equals one when edge {u,v} is cut, this satisfies contiguity?

# objective is to minimize cut edges
m.setObjective( gp.quicksum( y[u,v] for u,v in G.edges ), GRB.MINIMIZE )


Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2417480
Academic license 2417480 - for non-commercial use only - registered to um___@ttu.edu


In [6]:
# add constraints saying that each county i is assigned to one district
m.addConstrs( gp.quicksum(x[i,j] for j in range(k)) == 1 for i in G.nodes)

# add constraints saying that each district has population at least L and at most U
m.addConstrs( gp.quicksum( G.nodes[i]['POP100'] * x[i,j] for i in G.nodes) >= L for j in range(k) )
m.addConstrs( gp.quicksum( G.nodes[i]['POP100'] * x[i,j] for i in G.nodes) <= U for j in range(k) )

# add constraints saying that edge {u,v} is cut if u is assigned to district j but v is not.
m.addConstrs( x[u,j] - x[v,j] <= y[u,v] for u,v in G.edges for j in range(k))

m.update()

In [7]:
# Now, let's add contiguity constraints and re-solve the model.
# We will use the contiguity constraints of Hojny et al. (MPC, 2021)
#   https://link.springer.com/article/10.1007/s12532-020-00186-3

# Add root variables: r[i,j] equals 1 if node i is the "root" of district j
r = m.addVars( G.nodes, k, vtype=GRB.BINARY )

# Add flow variables: f[u,v] = amount of flow sent across arc uv 
#  Flows are sent across arcs of the directed version of G which we call DG

import networkx as nx
DG = nx.DiGraph(G)      # directed version of G

f = m.addVars( DG.edges )

In [8]:
# The big-M proposed by Hojny et al.
M = G.number_of_nodes() - k + 1

# Each district j should have one root
m.addConstrs( gp.quicksum( r[i,j] for i in G.nodes ) == 1 for j in range(k) )

# If node i is not assigned to district j, then it cannot be its root
m.addConstrs( r[i,j] <= x[i,j] for i in G.nodes for j in range(k) ) 

# if not a root, consume some flow.
# if a root, only send out (so much) flow.
m.addConstrs( gp.quicksum( f[j,i] - f[i,j] for j in G.neighbors(i) ) 
             >= 1 - M * gp.quicksum( r[i,j] for j in range(k) ) for i in G.nodes )

# do not send flow across cut edges
m.addConstrs( f[i,j] + f[j,i] <= M * ( 1 - y[i,j] ) for i,j in G.edges )

m.update()


In [9]:
# OPtimize to solve LP
m.optimize()

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Academic license 2417480 - for non-commercial use only - registered to um___@ttu.edu
Optimize a model with 1422 rows, 1176 columns and 5748 nonzeros
Model fingerprint: 0xa1a4a853
Variable types: 384 continuous, 792 integer (792 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+05]
Presolve time: 0.01s
Presolved: 1422 rows, 1176 columns, 5748 nonzeros
Variable types: 384 continuous, 792 integer (792 binary)

Root relaxation: objective 0.000000e+00, 581 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0  316          -    0.00000      -     -    0s


In [10]:
print("The number of cut edges is",m.objval)

# retrieve the districts and their populations
districts = [ [i for i in G.nodes if x[i,j].x > 0.5] for j in range(k)]
district_counties = [ [ G.nodes[i]["NAMELSAD20"] for i in districts[j] ] for j in range(k)]
district_populations = [ sum(G.nodes[i]["POP100"] for i in districts[j]) for j in range(k) ]

# print district info
for j in range(k):
    print("District",j,"has population",district_populations[j],"and contains counties",district_counties[j])
    print("")

The number of cut edges is 33.0
District 0 has population 751754 and contains counties ['Franklin County', 'Crawford County', 'Benton County', 'Madison County', 'Sebastian County', 'Washington County']

District 1 has population 754547 and contains counties ['Little River County', 'Ashley County', 'Desha County', 'Montgomery County', 'Howard County', 'Nevada County', 'Grant County', 'Dallas County', 'Cleveland County', 'Lafayette County', 'Chicot County', 'Pope County', 'Bradley County', 'Drew County', 'Pike County', 'Union County', 'Hempstead County', 'Polk County', 'Clark County', 'Logan County', 'Miller County', 'Arkansas County', 'Johnson County', 'Garland County', 'Sevier County', 'Jefferson County', 'Lincoln County', 'Scott County', 'Hot Spring County', 'Columbia County', 'Ouachita County', 'Yell County', 'Calhoun County', 'Phillips County']

District 2 has population 754435 and contains counties ['Jackson County', 'Clay County', 'Baxter County', 'Boone County', 'St. Francis Coun

In [16]:
# To read the arkansas county shape file 
import geopandas as gpd
filepath = '/Users/ujwalkumarmellacheruvu/Public/REDISTRICTING'
filename = 'AR_county.shp'
New_path = os.path.join(filepath,filename)

# Read geopandas dataframe from file
df = gpd.read_file(New_path)

ImportError: the 'read_file' function requires the 'fiona' package, but it is not installed or does not import correctly.
Importing fiona resulted in: dlopen(/Users/ujwalkumarmellacheruvu/anaconda3/lib/python3.11/site-packages/fiona/_env.cpython-311-darwin.so, 0x0002): Library not loaded: @rpath/libtiff.5.dylib
  Referenced from: <79CD5E81-11CF-3606-B3E3-BEA9CE2708B3> /Users/ujwalkumarmellacheruvu/anaconda3/lib/libpoppler.126.0.0.dylib
  Reason: tried: '/Users/ujwalkumarmellacheruvu/anaconda3/lib/libtiff.5.dylib' (no such file), '/Users/ujwalkumarmellacheruvu/anaconda3/lib/libtiff.5.dylib' (no such file), '/Users/ujwalkumarmellacheruvu/anaconda3/lib/python3.11/site-packages/fiona/../../../libtiff.5.dylib' (no such file), '/Users/ujwalkumarmellacheruvu/anaconda3/lib/python3.11/site-packages/fiona/../../../libtiff.5.dylib' (no such file), '/Users/ujwalkumarmellacheruvu/anaconda3/bin/../lib/libtiff.5.dylib' (no such file), '/Users/ujwalkumarmellacheruvu/anaconda3/bin/../lib/libtiff.5.dyli